In [269]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [270]:
# --- 1. Load Datasets ---
df1 = pd.read_csv("Dataset/completeSpamAssassin.csv")
df2 = pd.read_csv("Dataset/emails.csv")
df3 = pd.read_csv("Dataset/enronSpamSubset.csv")
df4 = pd.read_csv("Dataset/lingSpam.csv")
df5 = pd.read_csv("Dataset/spam mail.csv")

In [271]:
dfs = [df1, df2, df3, df4, df5]

In [272]:
# --- 2. Drop unnamed columns safely ---
for idx, df in enumerate(dfs, start=1):
    unnamed_cols = [col for col in df.columns if col.startswith('Unnamed')]
    if unnamed_cols:
        df.drop(columns=unnamed_cols, inplace=True)
        print(f"✅ Dropped {unnamed_cols} from df{idx}")
    else:
        print(f"ℹ️ No unnamed columns found in df{idx}")

✅ Dropped ['Unnamed: 0'] from df1
ℹ️ No unnamed columns found in df2
✅ Dropped ['Unnamed: 0.1', 'Unnamed: 0'] from df3
✅ Dropped ['Unnamed: 0'] from df4
ℹ️ No unnamed columns found in df5


In [273]:
# --- 3. Standardize column names ---
df2.rename(columns={'text': 'Body', 'spam': 'Label'}, inplace=True)
df5.rename(columns={'Category': 'Label', 'Masseges': 'Body'}, inplace=True)

In [274]:
# --- 4. Convert text labels to numeric ---
df5["Label"] = df5["Label"].replace({"ham": 0, "spam": 1})

C:\Users\nikhi\AppData\Local\Temp\ipykernel_15792\1075495117.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df5["Label"] = df5["Label"].replace({"ham": 0, "spam": 1})


In [275]:
# --- 5. Clean “Subject:” prefix if exists ---
for df in [df3, df4]:
    if "Body" in df.columns:
        df["Body"] = df["Body"].astype(str).str.replace("Subject:", "", regex=False).str.strip()


In [276]:
# --- 6. Keep only Body + Label columns where available ---
for i, df in enumerate(dfs):
    if all(col in df.columns for col in ["Body", "Label"]):
        dfs[i] = df[["Body", "Label"]]

In [277]:
# --- 7. Merge all into one big DataFrame ---
dfs = pd.concat(dfs, ignore_index=True)
print(f"✅ Combined dataset shape: {dfs.shape}")

✅ Combined dataset shape: (29951, 2)


In [278]:
dfs["Label"].unique()

array([1, 0])

In [279]:
# --- 8. Drop missing & duplicate rows ---
dfs.dropna(inplace=True)
dfs.drop_duplicates(inplace=True)
print(f"✅ After cleaning: {dfs.shape}")

✅ After cleaning: (28434, 2)


In [280]:
# --- 9. Text preprocessing ---
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [281]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [282]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)  # remove URLs
    text = re.sub(r'[^a-z\s]', ' ', text)       # remove non-letter characters
    text = re.sub(r'\s+', ' ', text).strip()    # remove extra spaces
    words = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return ' '.join(words)

In [283]:
dfs['Body'] = dfs['Body'].apply(clean_text)
dfs = dfs[dfs['Body'].str.len() > 3]

In [284]:
# --- 10. Quality checks ---
dfs['word_count'] = dfs['Body'].apply(lambda x: len(x.split()))
print("\n📊 Word count stats:\n", dfs['word_count'].describe())
print("\n📈 Label distribution:\n", dfs['Label'].value_counts(normalize=True))


📊 Word count stats:
 count    2.839700e+04
mean     1.854568e+02
std      8.703130e+03
min      1.000000e+00
25%      2.100000e+01
50%      6.100000e+01
75%      1.410000e+02
max      1.465726e+06
Name: word_count, dtype: float64

📈 Label distribution:
 Label
0    0.697996
1    0.302004
Name: proportion, dtype: float64


In [285]:
# --- 11. Final check ---
print("\n✅ Final dataset sample:")
print(dfs.head())
print(f"\n✅ Final dataset shape: {dfs.shape}")


✅ Final dataset sample:
                                                Body  Label  word_count
0  save life insurance spend life quote saving en...      1          95
1  fight risk cancer slim guaranteed lose lb day ...      1          52
2  fight risk cancer slim guaranteed lose lb day ...      1          37
3  adult club offer free membership instant acces...      1         207
4  thought might like slim guaranteed lose lb day...      1          44

✅ Final dataset shape: (28397, 3)


In [286]:
# --- 12. Save cleaned dataset ---
dfs.to_csv("cleaned_spam_dataset.csv", index=False, encoding='utf-8')
print(f"✅ Saved cleaned dataset as 'cleaned_spam_dataset.csv' with {dfs.shape[0]} rows.")


✅ Saved cleaned dataset as 'cleaned_spam_dataset.csv' with 28397 rows.
